<a href="https://colab.research.google.com/github/avani28/Natural-Disaster-Detection-System-using-Deep-Learning-and-Computer-Vision-/blob/main/Natural_Disaster_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Natural Disaster Detection System**

**Load the Dataset**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


**Import Necessary Packages**

In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import sys
import os
import tempfile
from imutils import paths
from collections import deque
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from google.colab.patches import cv2_imshow

***Cyclical Learning Rate Callback***

In [ ]:
class CyclicLR(Callback):
	
	def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
				 gamma=1., scale_fn=None, scale_mode='cycle'):
		super(CyclicLR, self).__init__()

		self.base_lr = base_lr
		self.max_lr = max_lr
		self.step_size = step_size
		self.mode = mode
		self.gamma = gamma
		if scale_fn == None:
			if self.mode == 'triangular':
				self.scale_fn = lambda x: 1.
				self.scale_mode = 'cycle'
			elif self.mode == 'triangular2':
				self.scale_fn = lambda x: 1 / (2. ** (x - 1))
				self.scale_mode = 'cycle'
			elif self.mode == 'exp_range':
				self.scale_fn = lambda x: gamma ** (x)
				self.scale_mode = 'iterations'
		else:
			self.scale_fn = scale_fn
			self.scale_mode = scale_mode
		self.clr_iterations = 0.
		self.trn_iterations = 0.
		self.history = {}

		self._reset()

	def _reset(self, new_base_lr=None, new_max_lr=None,
			   new_step_size=None):
		if new_base_lr != None:
			self.base_lr = new_base_lr
		if new_max_lr != None:
			self.max_lr = new_max_lr
		if new_step_size != None:
			self.step_size = new_step_size
		self.clr_iterations = 0.

	def clr(self):
		cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
		x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
		if self.scale_mode == 'cycle':
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
		else:
			return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
				self.clr_iterations)

	def on_train_begin(self, logs={}):
		logs = logs or {}

		if self.clr_iterations == 0:
			K.set_value(self.model.optimizer.lr, self.base_lr)
		else:
			K.set_value(self.model.optimizer.lr, self.clr())

	def on_batch_end(self, epoch, logs=None):

		logs = logs or {}
		self.trn_iterations += 1
		self.clr_iterations += 1

		self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
		self.history.setdefault('iterations', []).append(self.trn_iterations)

		for k, v in logs.items():
			self.history.setdefault(k, []).append(v)

		K.set_value(self.model.optimizer.lr, self.clr())





***Learning Rate Finder***

In [ ]:
class LearningRateFinder:
	def __init__(self, model, stopFactor=4, beta=0.98):
		
		self.model = model
		self.stopFactor = stopFactor
		self.beta = beta

		self.lrs = []
		self.losses = []

		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def reset(self):
		
		self.lrs = []
		self.losses = []
		self.lrMult = 1
		self.avgLoss = 0
		self.bestLoss = 1e9
		self.batchNum = 0
		self.weightsFile = None

	def is_data_iter(self, data):
		
		iterClasses = ["NumpyArrayIterator", "DirectoryIterator",
			 "DataFrameIterator", "Iterator", "Sequence"]

		
		return data.__class__.__name__ in iterClasses

	def on_batch_end(self, batch, logs):
		lr = K.get_value(self.model.optimizer.lr)
		self.lrs.append(lr)

		
		l = logs["loss"]
		self.batchNum += 1
		self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta) * l)
		smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
		self.losses.append(smooth)

		
		stopLoss = self.stopFactor * self.bestLoss

		
		if self.batchNum > 1 and smooth > stopLoss:
			
			self.model.stop_training = True
			return

		
		if self.batchNum == 1 or smooth < self.bestLoss:
			self.bestLoss = smooth

		
		lr *= self.lrMult
		K.set_value(self.model.optimizer.lr, lr)

	def find(self, trainData, startLR, endLR, epochs=None,
		stepsPerEpoch=None, batchSize=32, sampleSize=2048,
		verbose=1):
		
		self.reset()

		
		useGen = self.is_data_iter(trainData)

		
		if useGen and stepsPerEpoch is None:
			msg = "Using generator without supplying stepsPerEpoch"
			raise Exception(msg)

		elif not useGen:
			numSamples = len(trainData[0])
			stepsPerEpoch = np.ceil(numSamples / float(batchSize))

		if epochs is None:
			epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

		numBatchUpdates = epochs * stepsPerEpoch

		self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

		self.weightsFile = tempfile.mkstemp()[1]
		self.model.save_weights(self.weightsFile)

		origLR = K.get_value(self.model.optimizer.lr)
		K.set_value(self.model.optimizer.lr, startLR)

		callback = LambdaCallback(on_batch_end=lambda batch, logs:
			self.on_batch_end(batch, logs))

		if useGen:
			self.model.fit_generator(
				trainData,
				steps_per_epoch=stepsPerEpoch,
				epochs=epochs,
				verbose=verbose,
				callbacks=[callback])

		else:
			self.model.fit(
				trainData[0], trainData[1],
				batch_size=batchSize,
				epochs=epochs,
				callbacks=[callback],
				verbose=verbose)

		self.model.load_weights(self.weightsFile)
		K.set_value(self.model.optimizer.lr, origLR)

	def plot_loss(self, skipBegin=10, skipEnd=1, title=""):
		lrs = self.lrs[skipBegin:-skipEnd]
		losses = self.losses[skipBegin:-skipEnd]

		plt.plot(lrs, losses)
		plt.xscale("log")
		plt.xlabel("Learning Rate (Log Scale)")
		plt.ylabel("Loss")

		if title != "":
			plt.title(title)


**Set the Path to Dataset**

In [ ]:
DATASET_PATH = '/gdrive/MyDrive/Cyclone_Wildfire_Flood_Earthquake_Database'
Cyclone='/gdrive/MyDrive/Cyclone_Wildfire_Flood_Earthquake_Database/Cyclone'
Earthquake='/gdrive/MyDrive/Cyclone_Wildfire_Flood_Earthquake_Database/Earthquake'
Wildfire="/gdrive/MyDrive/Cyclone_Wildfire_Flood_Earthquake_Database/Flood"
Flood="/gdrive/MyDrive/Cyclone_Wildfire_Flood_Earthquake_Database/Wildfire"

# initializing the class labels in dataset
CLASSES = ["Cyclone", "Earthquake", "Flood", "Wildfire"]


**Set the Hyperparameter Values**

In [ ]:
MIN_LR = 1e-6  #minimum learning rate
MAX_LR = 1e-4  #maximum learning rate
BATCH_SIZE = 32  #batch size
STEP_SIZE = 8  #step size
CLR_METHOD = "triangular"  #Cyclical Learning Rate Method
NUM_EPOCHS = 48  #number of epochs


**Set the Path to Saved Model and Output Curves**

In [ ]:
output = '/content/output'
MODEL_PATH = os.path.sep.join(["/content/output", "natural_disaster.model"])

LRFIND_PLOT_PATH = os.path.sep.join(["/content/output", "lrfind_plot.png"])
TRAINING_PLOT_LOSS_PATH = os.path.sep.join(["/content/output", "training_plot_loss.png"])
CLR_PLOT_PATH = os.path.sep.join(["/content/output", "clr_plot.png"])
TRAINING_PLOT_ACCURACY_PATH = os.path.sep.join(["/content/output", "training_plot_accuracy.png"])
CONFUSION_MATRIX_PATH =  os.path.sep.join(["/content/output", "confusion_matrix.png"])


**Data Splitting and Image Preprocessing**

In [ ]:
#define train,test,validation split ratio
TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.1
TEST_SPLIT = 0.25

print("Loading images...")
imagePaths = list(paths.list_images(DATASET_PATH))
data = []
labels = []

for imagePath in imagePaths:
	label = imagePath.split(os.path.sep)[-2]

	image = cv2.imread(imagePath)  #load the image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #convert it to RGB channel ordering
	image = cv2.resize(image, (224, 224))  # resize it to be a fixed 224x224 pixels, ignoring aspect ratio

	data.append(image)
	labels.append(label)

print("processing images...")
data = np.array(data, dtype="float32")
labels = np.array(labels)
 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=TEST_SPLIT, random_state=42)

# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
	test_size=VAL_SPLIT, random_state=84)

# initialize the training data augmentation object
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")


Loading images...
processing images...


**Load VGG16 Network**

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

58892288/58889256 [==============================] - 0s 0us/step


**Build the Model**

In [ ]:
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

**Compile Model**

In [ ]:
print("Compiling model...")
opt = SGD(lr=MIN_LR, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

Compiling model...


**Find Learning Rate**

In [ ]:
print("Finding learning rate...")
lrf = LearningRateFinder(model)
lrf.find(
	aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
		1e-10, 1e+1,
		stepsPerEpoch=np.ceil((trainX.shape[0] / float(BATCH_SIZE))),
		epochs=20,
		batchSize=BATCH_SIZE)
 
lrf.plot_loss()
plt.savefig(LRFIND_PLOT_PATH)
 
print("Learning rate finder complete")

stepSize = STEP_SIZE * (trainX.shape[0] // BATCH_SIZE)
clr = CyclicLR(
	mode=CLR_METHOD,
	base_lr=MIN_LR,
	max_lr=MAX_LR,
	step_size=stepSize)

Finding learning rate...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
94/94 [==============================] - 41s 349ms/step - loss: 19.5369 - accuracy: 0.2347
Epoch 2/20
94/94 [==============================] - 33s 350ms/step - loss: 18.9204 - accuracy: 0.2279
Epoch 3/20
94/94 [==============================] - 32s 342ms/step - loss: 19.2384 - accuracy: 0.2221
Epoch 4/20
94/94 [==============================] - 32s 341ms/step - loss: 19.5485 - accuracy: 0.2276
Epoch 5/20
94/94 [==============================] - 32s 342ms/step - loss: 19.5789 - accuracy: 0.2152
Epoch 6/20
94/94 [==============================] - 32s 341ms/step - loss: 17.5510 - accuracy: 0.2431
Epoch 7/20
94/94 [==============================] - 32s 344ms/step - loss: 16.4734 - accuracy: 0.2629
Epoch 8/20
94/94 [==============================] - 32s 341ms/step - loss: 12.7383 - accuracy: 0.3335
Epoch 9/20
94/94 [==============================] - 32s 341ms/step - loss: 8.1487 - accuracy: 0.4867
Epoch 10/20
94/94 [==============================] - 32s 341ms/step - loss: 4.1989 

**Train the Network/Fit the Model**

In [ ]:
print("Training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
	validation_data=(valX, valY),
	steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
	epochs=NUM_EPOCHS,
	callbacks=[clr],
	verbose=1)
print("Network trained")

Training network...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/48
93/93 [==============================] - 35s 372ms/step - loss: 256.6813 - accuracy: 0.3667 - val_loss: 62.7711 - val_accuracy: 0.6066
Epoch 2/48
93/93 [==============================] - 33s 355ms/step - loss: 14.7309 - accuracy: 0.7033 - val_loss: 17.0962 - val_accuracy: 0.7808
Epoch 3/48
93/93 [==============================] - 33s 356ms/step - loss: 5.6595 - accuracy: 0.7886 - val_loss: 6.1854 - val_accuracy: 0.8168
Epoch 4/48
93/93 [==============================] - 33s 356ms/step - loss: 2.2090 - accuracy: 0.8129 - val_loss: 2.2436 - val_accuracy: 0.8468
Epoch 5/48
93/93 [==============================] - 33s 358ms/step - loss: 1.4867 - accuracy: 0.8288 - val_loss: 1.5171 - val_accuracy: 0.8589
Epoch 6/48
93/93 [==============================] - 33s 356ms/step - loss: 1.2211 - accuracy: 0.8396 - val_loss: 1.8892 - val_accuracy: 0.8318
Epoch 7/48
93/93 [==============================] - 33s 357ms/step - loss: 1.4116 - accuracy: 0.8484 - val_loss: 1.0082 - val_accuracy: 0

**Evaluate the Network**

In [ ]:
print("Evaluating network...")
predictions = model.predict(testX, batch_size=BATCH_SIZE)
print('Classification Report: ')
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=CLASSES))


Evaluating network...
Classification Report: 
              precision    recall  f1-score   support

     Cyclone       0.98      0.97      0.98       244
  Earthquake       0.97      0.91      0.94       328
       Flood       0.86      0.95      0.90       249
    Wildfire       0.96      0.95      0.96       286

    accuracy                           0.94      1107
   macro avg       0.94      0.95      0.94      1107
weighted avg       0.95      0.94      0.94      1107



**Save the Model to Disk**

In [ ]:
print("Serializing network to '{}'...".format(MODEL_PATH))
model.save(MODEL_PATH)

Serializing network to '/content/output/natural_disaster.model'...
INFO:tensorflow:Assets written to: /content/output/natural_disaster.model/assets


**Plot and Save Loss Curve** 

In [ ]:
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.show()
plt.savefig(TRAINING_PLOT_LOSS_PATH)

**Plot and Save Accuracy Curve**

In [ ]:
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig(TRAINING_PLOT_ACCURACY_PATH)
plt.show()

**Plot and Save Learning Rate History Curve**

In [ ]:
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")
plt.savefig(CLR_PLOT_PATH)
plt.show()

**Plot Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix1(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

y_test = testY.argmax(axis=1)
y_pred = predictions.argmax(axis=1)
lb = ["Cyclone", "Earthquake", "Flood", "Wildfire"] #Thunderstorm, Building_Collapse
# Plot normalized confusion matrix
plot_confusion_matrix1(y_test, y_pred, classes=lb, normalize=True,
                      title='Normalized confusion matrix')

plt.savefig(CONFUSION_MATRIX_PATH)

Normalized confusion matrix
[[0.97 0.   0.01 0.02]
 [0.   0.91 0.09 0.  ]
 [0.   0.03 0.95 0.02]
 [0.02 0.   0.02 0.95]]


In [ ]:
!pip install twilio
from twilio.rest import Client

     |████████████████████████████████| 460kB 8.7MB/s 
  Created wheel for twilio: filename=twilio-6.50.1-py2.py3-none-any.whl size=1208685 sha256=fd5534674045e08d4e5055fb1fb219c9b2867317d1e79a9909b403ed56d0fbfa
  Stored in directory: /root/.cache/pip/wheels/17/10/6c/1b04371d399b059dcea195e00729e096fd959e1e35b0e7c8a2
Successfully built twilio


**Predict the Video**

In [ ]:
from twilio.rest import Client
input='/gdrive/MyDrive/videos/cyclone_1.mp4'

size=128
display=1

# load the trained model from disk
print("Loading model and label binarizer...")
model = load_model(MODEL_PATH)

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=size)  #predictions queue

print("Processing video...")
vs = cv2.VideoCapture(input)  #initializing video stream
writer = None  #pointer to output video file
(W, H) = (None, None)  #intialize frame dimensions
 
client = Client("ACac0f843371e740077a4aa7734e4e2ad7", "4a5605c2a39d5b18dc0383f68b6b7415")
prelabel = ''
ok = 'Normal'
fi_label = []
framecount = 0

while True:
  (grabbed,frame)  = vs.read()

  if not grabbed:
    break
  if W is None or H is None:
    (H,W)=frame.shape[:2]
  framecount = framecount + 1

  output=frame.copy()
  frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
  frame=cv2.resize(frame,(224,224))
  frame=frame.astype("float32")
  frame=frame - mean

  preds = model.predict(np.expand_dims(frame,axis=0))[0]
  prediction=preds.argmax(axis=0)
  Q.append(preds)

  results = np.array(Q).mean(axis=0)
  maxprobab=np.max(results)
  i=np.argmax(results)
  label=CLASSES[i]

  rest = 1-maxprobab

  diff= (maxprobab)-(rest)
  th=100
  if diff>0.80:
    th=diff
    fi_label = np.append(fi_label, label)
    text = "Alert : {} - {:.2f}%".format((label), maxprobab * 100)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

    if label != prelabel:
      client.messages \
      .create(to="+911234567890", 
            from_="+19388882407", 
            body='\n'+ str(text))
    prelabel = label

  if writer is None:
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter('/content/output/result.mp4', fourcc, 30,(W, H), True)

  writer.write(output)

print('Frame count', framecount)
print('Count label', fi_label)	
#cv2_imshow(output)
writer.release()
vs.release()

Loading model and label binarizer...
Processing video...
Frame count 436
Count label ['Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone'
 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'Cyclone' 'C